## Prepare the inputs

- encoder_input_data: 2D array of shape `(num_images * 5, 512)`
- decoder_input_data: 3D array of shape `(num_captions, max_words_in_sentence, num_words)`
- decoder_output_data: same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]

## Image Preprocessing

obtain bottleneck features

In [22]:
import numpy as np

In [23]:
# Since there are 5 captions per image, duplicate the bottleneck features
def duplicate_bottleneck_features(features):
    num_captions = 5 # 5 stands for number of captions per image
    num_rows = features.shape[0] * num_captions 

    features_dup = np.zeros((num_rows, features.shape[1]))
    for i, image in enumerate(features):
        for j in range(num_captions):
            features_dup[i*num_captions + j] = image
    return features_dup    

In [24]:
bottleneck_features = np.load('bottleneck_features/Flicker8k_bottleneck_features_VGG16_avgpooling.npz')
bottleneck_features_train = bottleneck_features["train"]
bottleneck_features_validation = bottleneck_features["validation"]
bottleneck_features_test = bottleneck_features["test"]

bottleneck_features_train_dup = duplicate_bottleneck_features(bottleneck_features_train)
bottleneck_features_validation_dup = duplicate_bottleneck_features(bottleneck_features_validation)
bottleneck_features_test_dup = duplicate_bottleneck_features(bottleneck_features_test)

In [25]:
print(bottleneck_features_train_dup.shape)
print(bottleneck_features_validation_dup.shape)
print(bottleneck_features_test_dup.shape)

(30000, 512)
(5000, 512)
(5000, 512)


## Word Embedding

In [26]:
from caption_utils import *
train_fns_list, dev_fns_list, test_fns_list = load_split_lists()

train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)     
captions_data = (train_captions_raw.copy(), dev_captions_raw.copy(), test_captions_raw.copy())
train_captions, dev_captions, test_captions = process_captions(captions_data, token2idx)

In [27]:
print(len(vocab))

2531


## Caption Preprocessing


In [28]:
captions_onehot = np.load('preprocessed_captions/Flicker8k_onehot_'+str(len(vocab))+'_words.npz')
train_captions_onehot = captions_onehot["train"]
validation_captions_onehot = captions_onehot["validation"]
test_captions_onehot = captions_onehot["test"]

train_captions_onehot = train_captions_onehot.astype(np.float32)
validation_captions_onehot = validation_captions_onehot.astype(np.float32)
test_captions_onehot = test_captions_onehot.astype(np.float32)

In [29]:
print(train_captions_onehot.shape)
print(validation_captions_onehot.shape)
print(test_captions_onehot.shape)

(30000, 39, 2531)
(5000, 39, 2531)
(5000, 39, 2531)


In [30]:
print(train_captions_onehot.dtype, validation_captions_onehot.dtype, test_captions_onehot.dtype)

float32 float32 float32


In [31]:
def captions_onehot_split(captions_onehot):
    """ returns decoder input data and decoder target data """
    return captions_onehot[:, :-1, :], captions_onehot[:, :, :]

### Training captions - > decoder input, target data  

In [32]:
train_decoder_input, train_decoder_target = captions_onehot_split(train_captions_onehot)
validation_decoder_input, validation_decoder_target = captions_onehot_split(validation_captions_onehot)
test_decoder_input, test_decoder_target = captions_onehot_split(test_captions_onehot)

In [33]:
train_encoder_output = bottleneck_features_train_dup.astype(np.float32)
test_encoder_output = bottleneck_features_test_dup.astype(np.float32)
validation_encoder_output = bottleneck_features_validation_dup.astype(np.float32)

In [34]:
test_decoder_input = np.argmax(test_decoder_input, axis=-1)
train_decoder_input = np.argmax(train_decoder_input, axis=-1)
validation_decoder_input = np.argmax(validation_decoder_input, axis=-1)

In [35]:
print("Decoder Input", train_decoder_input.shape, train_decoder_input.dtype)
print("Decoder Target", train_decoder_target.shape, train_decoder_target.dtype)
print("Encoder Output", train_encoder_output.shape, train_encoder_output.dtype)

Decoder Input (30000, 38) int64
Decoder Target (30000, 39, 2531) float32
Encoder Output (30000, 512) float32


## Build Model

In [43]:
batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 300  # Latent dimensionality of the encoding space.
num_samples = train_encoder_output.shape[0]

-------------------------
# Experiment 

## Parameters

In [44]:
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, GRU, Dense, Embedding, BatchNormalization, RepeatVector, Concatenate, TimeDistributed, Merge
from keras import backend as K
import tensorflow as tf

In [45]:
emb_size = 150
lstm_size = 300
vocab_size = len(vocab)
max_length = train_decoder_target.shape[1]
print(max_length)

39


In [46]:
K.clear_session()

# Image -> Image embedding
img_emb = Sequential()
img_emb.add(Dense(emb_size, input_dim=train_encoder_output.shape[1], activation='relu'))
img_emb.add(RepeatVector(1))

# Sentence to Word embedding
word_emb = Sequential()
word_emb.add(Embedding(input_dim=vocab_size, output_dim=emb_size, input_length=max_length-1))

# Merge img_emb and word_emb
seq_in = Sequential()
seq_in.add(Merge([img_emb, word_emb], mode='concat', concat_axis=1))
#seq_in.add(Concatenate([img_emb, word_emb]))

# RNN Layer
seq_in.add(GRU(lstm_size, return_sequences=True))
seq_in.add(TimeDistributed(Dense(vocab_size)))

seq_in.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

/home/ubuntu/anaconda3/envs/tensorflow-3.5/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


In [48]:
seq_in.fit([test_encoder_output, test_decoder_input], [test_decoder_target], epochs=20, batch_size=64, verbose=2)

Epoch 1/20
 - 7s - loss: 3.5304 - acc: 0.0049
Epoch 2/20
 - 7s - loss: 3.5637 - acc: 0.0049
Epoch 3/20
 - 6s - loss: 3.5191 - acc: 0.0049
Epoch 4/20
 - 7s - loss: 3.4645 - acc: 0.0049
Epoch 5/20
 - 6s - loss: 3.5300 - acc: 0.0049
Epoch 6/20
 - 6s - loss: 3.5017 - acc: 0.0049
Epoch 7/20
 - 7s - loss: 3.4441 - acc: 0.0049
Epoch 8/20
 - 7s - loss: 3.5877 - acc: 0.0064
Epoch 9/20
 - 7s - loss: 3.5056 - acc: 0.0109
Epoch 10/20
 - 6s - loss: 3.4462 - acc: 0.0133
Epoch 11/20
 - 6s - loss: 3.3879 - acc: 0.0127
Epoch 12/20
 - 6s - loss: 3.4527 - acc: 0.0121
Epoch 13/20
 - 6s - loss: 3.5179 - acc: 0.0120
Epoch 14/20
 - 7s - loss: 3.3859 - acc: 0.0120
Epoch 15/20
 - 7s - loss: 3.5151 - acc: 0.0119
Epoch 16/20
 - 7s - loss: 3.4381 - acc: 0.0119
Epoch 17/20
 - 7s - loss: 3.3823 - acc: 0.0119
Epoch 18/20
 - 6s - loss: 3.4420 - acc: 0.0119
Epoch 19/20
 - 6s - loss: 3.4796 - acc: 0.0119
Epoch 20/20
 - 7s - loss: 3.4317 - acc: 0.0119


In [41]:
print(img_emb.summary())
print(word_emb.summary())
print(seq_in.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 150)               76950     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 1, 150)            0         
Total params: 76,950
Trainable params: 76,950
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 38, 150)           379650    
Total params: 379,650
Trainable params: 379,650
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 39, 150)           0

In [42]:
from keras.utils import plot_model
plot_model(seq_in, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [ ]:
print(Merge.__doc__)